In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from flask import Flask, render_template, request, send_from_directory
import os
from PIL import Image, ImageDraw, ImageFont
import math
from math import pi

app = Flask(__name__)


brightness_cache = {}

def quantum(inp_path, out_path, block_size, font_size, magnitude, ascii_set, use_color=True):
    try:
        input_image = Image.open(inp_path)
    except FileNotFoundError:
        print("Image not found.")
        return

    input_width, input_height = input_image.size
    rows = input_height // block_size
    cols = input_width // block_size

    # Resize image for faster processing
    resized = input_image.resize((cols, rows), Image.Resampling.LANCZOS)

    font = ImageFont.load_default()


    bbox = font.getbbox("A")
    char_width = bbox[2] - bbox[0]
    char_height = bbox[3] - bbox[1]

    output_width = cols * char_width
    output_height = rows * char_height
    output = Image.new("RGB", (output_width, output_height), "black")
    draw = ImageDraw.Draw(output)

    simulator = Aer.get_backend('qasm_simulator')

    def get_brightness(val):
        if val in brightness_cache:
            return brightness_cache[val]

        num_qubits = 2
        qc = QuantumCircuit(num_qubits, num_qubits)
        theta = (val / 255.0) * (pi / 2)

        qc.ry(theta, 0)
        qc.ry(theta, 1)

        if magnitude > 0:
            qc.h(0)
            qc.cry((val / 255.0) * (pi / 4), 0, 1)

        qc.measure(range(num_qubits), range(num_qubits))
        final = transpile(qc, simulator)
        result = simulator.run(final, shots=1).result()
        counts = result.get_counts(qc)

        if counts:
            key = list(counts.keys())[0]
            qval = int(key, 2) / (2**num_qubits - 1)
            qbright = qval * 255
            blended = int(val * (1 - magnitude) + qbright * magnitude)
            brightness_cache[val] = blended
            return blended
        return val

    for row in range(rows):
        for col in range(cols):
            r, g, b = resized.convert("RGB").getpixel((col, row))

            if use_color:
                r_q = get_brightness(r)
                g_q = get_brightness(g)
                b_q = get_brightness(b)
                fill = (r_q, g_q, b_q)
                bright = int(0.2126 * r + 0.7152 * g + 0.0722 * b)
            else:
                gray = int(0.2126 * r + 0.7152 * g + 0.0722 * b)
                bright = get_brightness(gray)
                fill = (bright, bright, bright)

            ascii_index = min(int((bright / 255) * (len(ascii_set) - 1)), len(ascii_set) - 1)
            char = ascii_set[ascii_index]

            draw.text((col * char_width, row * char_height), char, font=font, fill=fill)

    output.save(out_path)
    print("Saved:", out_path)
    return output


if __name__ == "__main__":
    ascii_chars = " .'`^\",:;Il!i><~+_-?][}{1)(|/tfjrxnuvczXYUJCLQ0OZmwqpdbkhao*#MW&8%B@$"
    quantum(
        inp_path="testing.jpg",
        out_path="result_v1.jpg",
        block_size=6,
        font_size=10,
        magnitude=0.0000000001,
        ascii_set=ascii_chars,
        use_color=True
    )

Saved: result_v1.jpg
